# notre appli

Rendu le 16 févriller

In [1]:
import dash
import dash_bootstrap_components as dbc
from dash import Input, Output, State, dcc, html, Dash,dcc,html,callback
import pandas as pd 
import plotly.express as px
import Functions_dash as Fun
import Figure as fig
import pages as pag
from PIL import Image

In [2]:
# link fontawesome to get the chevron icons
app = dash.Dash(
    external_stylesheets=[dbc.themes.BOOTSTRAP, dbc.icons.FONT_AWESOME],
    suppress_callback_exceptions=True # If components are constructed by callbacks
)

# Bar noir en haut de page
navbar = dbc.NavbarSimple(
    children=[
        dbc.Button("Menu", outline=True, color="secondary", className="mr-1", id="btn_sidebar"), # Bouton pour le menu
    ],
    brand="",
    brand_href="#",
    color="dark",
    dark=True,
    fluid=True,
    links_left= True,
)

# Menus déplient 1
submenu_1 = [
    html.Li(
        # use Row and Col components to position the chevrons
        dbc.Row(
            [
                dbc.Col("Menu 1"),
                dbc.Col(
                    html.I(className="fas fa-chevron-right me-3"),
                    width="auto",
                ),
            ],
            className="my-1",
        ),
        style={"cursor": "pointer"},
        id="submenu-1",
    ),
    # we use the Collapse component to hide and reveal the navigation links
    dbc.Collapse(
        [
            dbc.NavLink("Evolution temporelle", href="/page-main"), # Page d'acceuil
            dbc.NavLink("Situation usager", href="/page-user"),
             dbc.NavLink("Carte des accidents en France", href="/page-map"), # Map page
        ],
        id="submenu-1-collapse",
    ),
]

# Menu déplient 2
submenu_2 = [
    html.Li(
        dbc.Row(
            [
                dbc.Col("Menu 2"),
                dbc.Col(
                    html.I(className="fas fa-chevron-right me-3"),
                    width="auto",
                ),
            ],
            className="my-1",
        ),
        style={"cursor": "pointer"},
        id="submenu-2",
    ),
    dbc.Collapse(
        [
            dbc.NavLink("Page 2.1", href="/page-2/1"),
            dbc.NavLink("Page 2.2", href="/page-2/2"),
        ],
        id="submenu-2-collapse",
    ),
]

# Div contenent le menu déplient
sidebar = html.Div(
    [
        html.H2("DashBike", className="display-4"),
        html.Hr(),
        html.Img(src = Image.open("accident_bike.png"),style={"width": "60%"}),
        html.P(
            "Vous trouverez ici les différentes pages du dashboard", className="lead"
        ),
        dbc.Nav(submenu_1 + submenu_2, vertical=True,pills=True),
        html.Img(src=Image.open("roue.png"),style={"width": "60%",
                                                  "margin-top": '150%'}),
    ],
    style=fig.SIDEBAR_STYLE,
    id="sidebar",
)

# Div racine de la page
app.layout = html.Div(
    [
        dcc.Store(id='side_click'),
        dcc.Location(id="url"),
        navbar,
        sidebar,
        html.Div(id="page-content", style=fig.CONTENT_STYLE), # Elément qui contien la page principale
    ],
)

# Fonction qui appel toute les fonction callback qui sont dans le fichier Fonction_dash.py
Fun.get_callbacks(app)



if __name__ == "__main__":
    app.run_server(debug=True,jupyter_mode="external")

Dash app running on http://127.0.0.1:8050/


In [2]:
fig.data_filter(fig.data, 'lat', x_select=[0,10])

,Unnamed: 0.1,Unnamed: 0,Num_Acc,date,an,mois,jour,hrmn,dep,com,...,_infos_commune.code_epci,code_postal,nom_commune_postal,latitude,longitude,nom_commune,nom_commune_complet,num_dep,dep_name,region_name
27128,27128,27128,200900002247,2009-01-13,2009,janvier,mardi,10:37,68,68066,...,246800726.0,NaN,NaN,NaN,NaN,non renseigné,NaN,68,Haut-Rhin,Grand Est
27129,27129,27129,200900002272,2009-01-27,2009,janvier,mardi,14:00,68,68298,...,246800395.0,NaN,NaN,NaN,NaN,non renseigné,NaN,68,Haut-Rhin,Grand Est
27130,27130,27130,200900002301,2009-01-03,2009,janvier,samedi,18:15,14,14366,...,200069532.0,NaN,NaN,NaN,NaN,non renseigné,NaN,14,Calvados,Normandie
27131,27131,27131,200900002301,2009-01-03,2009,janvier,samedi,18:15,14,14366,...,200069532.0,NaN,NaN,NaN,NaN,non renseigné,NaN,14,Calvados,Normandie
27229,27229,27229,200900003287,2009-01-22,2009,janvier,jeudi,10:15,30,30155,...,243000643.0,NaN,NaN,NaN,NaN,non renseigné,NaN,30,Gard,Occitanie
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92703,92703,92703,202100037919,2021-05-28,2021,mai,vendredi,19:39,973,97307,...,249730045.0,NaN,NaN,NaN,NaN,non renseigné,NaN,973,Guyane,Guyane
93402,93402,93402,202100046842,2021-03-24,2021,mars,mercredi,15:07,973,97309,...,249730045.0,NaN,NaN,NaN,NaN,non renseigné,NaN,973,Guyane,Guyane
93568,93568,93568,202100049433,2021-02-23,2021,février,mardi,19:45,973,97353,...,249730037.0,97353.0,REGINA,3.940918,-52.519927,Régina,Régina,973,Guyane,Guyane
93717,93717,93717,202100050831,2021-02-20,2021,février,samedi,21:10,973,97305,...,249730045.0,NaN,NaN,NaN,NaN,non renseigné,NaN,973,Guyane,Guyane


In [10]:
fig.data[(fig.data['lat'] > [0,10][0]) & (fig.data['lat'] < [0,10][1])]

,Unnamed: 0.1,Unnamed: 0,Num_Acc,date,an,mois,jour,hrmn,dep,com,...,_infos_commune.code_epci,code_postal,nom_commune_postal,latitude,longitude,nom_commune,nom_commune_complet,num_dep,dep_name,region_name
27128,27128,27128,200900002247,2009-01-13,2009,janvier,mardi,10:37,68,68066,...,246800726.0,NaN,NaN,NaN,NaN,non renseigné,NaN,68,Haut-Rhin,Grand Est
27129,27129,27129,200900002272,2009-01-27,2009,janvier,mardi,14:00,68,68298,...,246800395.0,NaN,NaN,NaN,NaN,non renseigné,NaN,68,Haut-Rhin,Grand Est
27130,27130,27130,200900002301,2009-01-03,2009,janvier,samedi,18:15,14,14366,...,200069532.0,NaN,NaN,NaN,NaN,non renseigné,NaN,14,Calvados,Normandie
27131,27131,27131,200900002301,2009-01-03,2009,janvier,samedi,18:15,14,14366,...,200069532.0,NaN,NaN,NaN,NaN,non renseigné,NaN,14,Calvados,Normandie
27229,27229,27229,200900003287,2009-01-22,2009,janvier,jeudi,10:15,30,30155,...,243000643.0,NaN,NaN,NaN,NaN,non renseigné,NaN,30,Gard,Occitanie
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92703,92703,92703,202100037919,2021-05-28,2021,mai,vendredi,19:39,973,97307,...,249730045.0,NaN,NaN,NaN,NaN,non renseigné,NaN,973,Guyane,Guyane
93402,93402,93402,202100046842,2021-03-24,2021,mars,mercredi,15:07,973,97309,...,249730045.0,NaN,NaN,NaN,NaN,non renseigné,NaN,973,Guyane,Guyane
93568,93568,93568,202100049433,2021-02-23,2021,février,mardi,19:45,973,97353,...,249730037.0,97353.0,REGINA,3.940918,-52.519927,Régina,Régina,973,Guyane,Guyane
93717,93717,93717,202100050831,2021-02-20,2021,février,samedi,21:10,973,97305,...,249730045.0,NaN,NaN,NaN,NaN,non renseigné,NaN,973,Guyane,Guyane


In [3]:
fig.fig_seri_reg(fig.data, [200, 800], [2010, 2015])

In [6]:
legend_groups = set()
for trace in fig.fig_seri_reg(fig.data)['data']:
    legend_groups.add(trace['legendgroup'])

In [7]:
print(legend_groups)

{"Provence-Alpes-Côte d'Azur", 'Corse', 'Grand Est', 'Pays de la Loire', 'Île-de-France', 'Guyane', 'Martinique', 'Mayotte', 'Bourgogne-Franche-Comté', 'Nouvelle-Aquitaine', 'Guadeloupe', 'Auvergne-Rhône-Alpes', 'Bretagne', 'La Réunion', 'Normandie', 'Occitanie', 'Centre-Val de Loire', 'Hauts-de-France'}


In [2]:
print(Fun.get_legend_groups(fig.fig_seri_reg(fig.data)))

{'Nouvelle-Aquitaine', 'Normandie', 'Mayotte', 'Guyane', 'Auvergne-Rhône-Alpes', "Provence-Alpes-Côte d'Azur", 'Occitanie', 'Bretagne', 'La Réunion', 'Corse', 'Guadeloupe', 'Hauts-de-France', 'Île-de-France', 'Bourgogne-Franche-Comté', 'Centre-Val de Loire', 'Pays de la Loire', 'Martinique', 'Grand Est'}


In [7]:
test = Fun.split_fig(fig.fig_seri_reg(fig.data))

In [9]:
test[1]

In [4]:
fig.data['an'].unique()

array([2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015,
       2016, 2017, 2018, 2019, 2020, 2021], dtype=int64)

In [7]:
type(fig.pie_age_grav('Blessé léger', 2005)['data'][0])

plotly.graph_objs._pie.Pie

In [5]:
fig.fig_seri_reg(fig.data)['data'][0]['customdata'][0][1]

'Auvergne-Rhône-Alpes'

In [5]:
fig.fig_seri_reg(fig.data)['data'][1]

Scatter({
    'customdata': array([[2005, 'Bourgogne-Franche-Comté', 528],
                         [2006, 'Bourgogne-Franche-Comté', 339],
                         [2007, 'Bourgogne-Franche-Comté', 434],
                         [2008, 'Bourgogne-Franche-Comté', 284],
                         [2009, 'Bourgogne-Franche-Comté', 291],
                         [2010, 'Bourgogne-Franche-Comté', 217],
                         [2011, 'Bourgogne-Franche-Comté', 228],
                         [2012, 'Bourgogne-Franche-Comté', 158],
                         [2013, 'Bourgogne-Franche-Comté', 159],
                         [2014, 'Bourgogne-Franche-Comté', 208],
                         [2015, 'Bourgogne-Franche-Comté', 241],
                         [2016, 'Bourgogne-Franche-Comté', 176],
                         [2017, 'Bourgogne-Franche-Comté', 172],
                         [2018, 'Bourgogne-Franche-Comté', 156],
                         [2019, 'Bourgogne-Franche-Comté', 253],
               

In [5]:
fig.fig_seri_reg(fig.data)['data'][0]['customdata']

array([[2005, 'Auvergne-Rhône-Alpes', 768],
       [2006, 'Auvergne-Rhône-Alpes', 762],
       [2007, 'Auvergne-Rhône-Alpes', 610],
       [2008, 'Auvergne-Rhône-Alpes', 594],
       [2009, 'Auvergne-Rhône-Alpes', 542],
       [2010, 'Auvergne-Rhône-Alpes', 521],
       [2011, 'Auvergne-Rhône-Alpes', 559],
       [2012, 'Auvergne-Rhône-Alpes', 568],
       [2013, 'Auvergne-Rhône-Alpes', 551],
       [2014, 'Auvergne-Rhône-Alpes', 583],
       [2015, 'Auvergne-Rhône-Alpes', 593],
       [2016, 'Auvergne-Rhône-Alpes', 568],
       [2017, 'Auvergne-Rhône-Alpes', 735],
       [2018, 'Auvergne-Rhône-Alpes', 538],
       [2019, 'Auvergne-Rhône-Alpes', 339],
       [2020, 'Auvergne-Rhône-Alpes', 332],
       [2021, 'Auvergne-Rhône-Alpes', 504]], dtype=object)

In [17]:
test = {
  "xaxis.autorange": "true",
  "xaxis.showspikes": "false",
  "yaxis.autorange": "true",
  "yaxis.showspikes": "false"
}

In [13]:
def relayoutData_transform(relayoutData):
    y_range = None
    x_range = None
        
    if 'autosize' in relayoutData:
        return None, None
        
    if 'xaxis.range[0]' in relayoutData or 'xaxis.range[1]' in relayoutData:
        x_range = [relayoutData['xaxis.range[0]'], relayoutData['xaxis.range[1]']]

    if 'yaxis.range[0]' in relayoutData or 'yaxis.range[1]' in relayoutData:
        y_range = [relayoutData['yaxis.range[0]'], relayoutData['yaxis.range[1]']]

    if "xaxis.autorange" in relayoutData:
         x_range = None

    if "yaxis.autorange" in relayoutData:
        y_range = None
        
    return x_range, y_range

In [18]:
relayoutData_transform(test)

(None, None)

In [24]:
fig.data[(fig.data['lat'] < -20) | (fig.data['lat'] > 0)]

,Unnamed: 0.1,Unnamed: 0,Num_Acc,date,an,mois,jour,hrmn,dep,com,...,_infos_commune.code_epci,code_postal,nom_commune_postal,latitude,longitude,nom_commune,nom_commune_complet,num_dep,dep_name,region_name
0,0,0,200500000030,2005-01-13,2005,janvier,jeudi,19:45,62,62331,...,200033579.0,NaN,NaN,NaN,NaN,non renseigné,NaN,62,Pas-de-Calais,Hauts-de-France
3,3,3,200500000093,2005-01-03,2005,janvier,lundi,13:30,02,02810,...,200071991.0,NaN,NaN,NaN,NaN,non renseigné,NaN,02,Aisne,Hauts-de-France
6,6,6,200500000260,2005-01-04,2005,janvier,mardi,16:45,22,22143,...,200068989.0,NaN,NaN,NaN,NaN,non renseigné,NaN,22,Côtes-d'Armor,Bretagne
10,10,10,200500000291,2005-01-24,2005,janvier,lundi,17:30,56,56155,...,244400610.0,NaN,NaN,NaN,NaN,non renseigné,NaN,56,Morbihan,Bretagne
14,14,14,200500000329,2005-01-04,2005,janvier,mardi,19:00,50,50213,...,NaN,NaN,NaN,NaN,NaN,non renseigné,NaN,50,Manche,Normandie
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94044,94044,94044,202100056508,2021-01-01,2021,janvier,vendredi,12:20,64,64400,...,246401756.0,64400.0,POEY D OLORON,43.246755,-0.663214,Poey-d'Oloron,Poey-d'Oloron,64,Pyrénées-Atlantiques,Nouvelle-Aquitaine
94045,94045,94045,202100056508,2021-01-01,2021,janvier,vendredi,12:20,64,64400,...,246401756.0,64400.0,PRECILHON,43.195778,-0.572974,Précilhon,Précilhon,64,Pyrénées-Atlantiques,Nouvelle-Aquitaine
94046,94046,94046,202100056508,2021-01-01,2021,janvier,vendredi,12:20,64,64400,...,246401756.0,64400.0,ST GOIN,43.231201,-0.725099,Saint-Goin,Saint-Goin,64,Pyrénées-Atlantiques,Nouvelle-Aquitaine
94047,94047,94047,202100056508,2021-01-01,2021,janvier,vendredi,12:20,64,64400,...,246401756.0,64400.0,SAUCEDE,43.265674,-0.669442,Saucède,Saucède,64,Pyrénées-Atlantiques,Nouvelle-Aquitaine


In [19]:
fig.data.columns.values

array(['Unnamed: 0.1', 'Unnamed: 0', 'Num_Acc', 'date', 'an', 'mois',
       'jour', 'hrmn', 'dep', 'com', 'lat', 'long', 'agg', 'int', 'col',
       'lum', 'atm', 'catr', 'circ', 'nbv', 'prof', 'plan', 'lartpc',
       'larrout', 'surf', 'infra', 'situ', 'grav', 'sexe', 'age',
       'trajet', 'secuexist', 'equipement', 'obs', 'obsm', 'choc', 'manv',
       'vehiculeid', 'typevehicules', 'manoeuvehicules', 'numVehicules',
       '_infos_commune.code_epci', 'code_postal', 'nom_commune_postal',
       'latitude', 'longitude', 'nom_commune', 'nom_commune_complet',
       'num_dep', 'dep_name', 'region_name'], dtype=object)

## Temp storage
### Number bar children

In [ ]:
dbc.Row(
                        [
                            dbc.Col(fig.nb_total, width=3),
                            dbc.Col(fig.nb_mort, width=3),
                            dbc.Col(fig.nb_hospital, width=3),
                            dbc.Col("Info 4", width=3),
                        ],
                    ),

In [6]:
fig.data[data]

,Country Name,Indicator Name,Year,Value
0,Arab World,"Agriculture, value added (% of GDP)",1962,NaN
1,Arab World,CO2 emissions (metric tons per capita),1962,0.760996
2,Arab World,Domestic credit provided by financial sector (...,1962,18.168690
3,Arab World,Electric power consumption (kWh per capita),1962,NaN
4,Arab World,Energy use (kg of oil equivalent per capita),1962,NaN
...,...,...,...,...
36955,Zimbabwe,"Industry, value added (% of GDP)",2007,33.074953
36956,Zimbabwe,"Inflation, GDP deflator (annual %)",2007,0.894887
36957,Zimbabwe,"Life expectancy at birth, total (years)",2007,44.177756
36958,Zimbabwe,Population density (people per sq. km of land ...,2007,34.374559


In [3]:
from dash import Dash, html, dcc, Input, Output, callback
import pandas as pd
import plotly.express as px

external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app = Dash(__name__, external_stylesheets=external_stylesheets)

df = pd.read_csv('https://plotly.github.io/datasets/country_indicators.csv')


app.layout = html.Div([
    html.Div([

        html.Div([
            dcc.Dropdown(
                df['Indicator Name'].unique(),
                'Fertility rate, total (births per woman)',
                id='crossfilter-xaxis-column',
            ),
            dcc.RadioItems(
                ['Linear', 'Log'],
                'Linear',
                id='crossfilter-xaxis-type',
                labelStyle={'display': 'inline-block', 'marginTop': '5px'}
            )
        ],
        style={'width': '49%', 'display': 'inline-block'}),

        html.Div([
            dcc.Dropdown(
                df['Indicator Name'].unique(),
                'Life expectancy at birth, total (years)',
                id='crossfilter-yaxis-column'
            ),
            dcc.RadioItems(
                ['Linear', 'Log'],
                'Linear',
                id='crossfilter-yaxis-type',
                labelStyle={'display': 'inline-block', 'marginTop': '5px'}
            )
        ], style={'width': '49%', 'float': 'right', 'display': 'inline-block'})
    ], style={
        'padding': '10px 5px'
    }),

    html.Div([
        dcc.Graph(
            id='crossfilter-indicator-scatter',
            hoverData={'points': [{'customdata': 'Japan'}]}
        )
    ], style={'width': '49%', 'display': 'inline-block', 'padding': '0 20'}),
    html.Div([
        dcc.Graph(id='x-time-series'),
        dcc.Graph(id='y-time-series'),
    ], style={'display': 'inline-block', 'width': '49%'}),

    html.Div(dcc.Slider(
        df['Year'].min(),
        df['Year'].max(),
        step=None,
        id='crossfilter-year--slider',
        value=df['Year'].max(),
        marks={str(year): str(year) for year in df['Year'].unique()}
    ), style={'width': '49%', 'padding': '0px 20px 20px 20px'})
])


@callback(
    Output('crossfilter-indicator-scatter', 'figure'),
    Input('crossfilter-xaxis-column', 'value'),
    Input('crossfilter-yaxis-column', 'value'),
    Input('crossfilter-xaxis-type', 'value'),
    Input('crossfilter-yaxis-type', 'value'),
    Input('crossfilter-year--slider', 'value'))
def update_graph(xaxis_column_name, yaxis_column_name,
                 xaxis_type, yaxis_type,
                 year_value):
    dff = df[df['Year'] == year_value]

    fig = px.scatter(x=dff[dff['Indicator Name'] == xaxis_column_name]['Value'],
            y=dff[dff['Indicator Name'] == yaxis_column_name]['Value'],
            hover_name=dff[dff['Indicator Name'] == yaxis_column_name]['Country Name']
            )

    fig.update_traces(customdata=dff[dff['Indicator Name'] == yaxis_column_name]['Country Name'])

    fig.update_xaxes(title=xaxis_column_name, type='linear' if xaxis_type == 'Linear' else 'log')

    fig.update_yaxes(title=yaxis_column_name, type='linear' if yaxis_type == 'Linear' else 'log')

    fig.update_layout(margin={'l': 40, 'b': 40, 't': 10, 'r': 0}, hovermode='closest')

    return fig


def create_time_series(dff, axis_type, title):

    fig = px.scatter(dff, x='Year', y='Value')

    fig.update_traces(mode='lines+markers')

    fig.update_xaxes(showgrid=False)

    fig.update_yaxes(type='linear' if axis_type == 'Linear' else 'log')

    fig.add_annotation(x=0, y=0.85, xanchor='left', yanchor='bottom',
                       xref='paper', yref='paper', showarrow=False, align='left',
                       text=title)

    fig.update_layout(height=225, margin={'l': 20, 'b': 30, 'r': 10, 't': 10})

    return fig


@callback(
    Output('x-time-series', 'figure'),
    Input('crossfilter-indicator-scatter', 'hoverData'),
    Input('crossfilter-xaxis-column', 'value'),
    Input('crossfilter-xaxis-type', 'value'))
def update_x_timeseries(hoverData, xaxis_column_name, axis_type):
    country_name = hoverData['points'][0]['customdata']
    dff = df[df['Country Name'] == country_name]
    dff = dff[dff['Indicator Name'] == xaxis_column_name]
    title = '<b>{}</b><br>{}'.format(country_name, xaxis_column_name)
    return create_time_series(dff, axis_type, title)


@callback(
    Output('y-time-series', 'figure'),
    Input('crossfilter-indicator-scatter', 'hoverData'),
    Input('crossfilter-yaxis-column', 'value'),
    Input('crossfilter-yaxis-type', 'value'))
def update_y_timeseries(hoverData, yaxis_column_name, axis_type):
    dff = df[df['Country Name'] == hoverData['points'][0]['customdata']]
    dff = dff[dff['Indicator Name'] == yaxis_column_name]
    return create_time_series(dff, axis_type, yaxis_column_name)


if __name__ == '__main__':
    app.run(debug=True)


In [10]:

dcc.Graph(figure=fig.fig_seri_reg(fig.data))[0]

Graph(figure=Figure({
    'data': [{'customdata': array([[2005, 'Auvergne-Rhône-Alpes', 768],
                                   [2006, 'Auvergne-Rhône-Alpes', 762],
                                   [2007, 'Auvergne-Rhône-Alpes', 610],
                                   [2008, 'Auvergne-Rhône-Alpes', 594],
                                   [2009, 'Auvergne-Rhône-Alpes', 542],
                                   [2010, 'Auvergne-Rhône-Alpes', 521],
                                   [2011, 'Auvergne-Rhône-Alpes', 559],
                                   [2012, 'Auvergne-Rhône-Alpes', 568],
                                   [2013, 'Auvergne-Rhône-Alpes', 551],
                                   [2014, 'Auvergne-Rhône-Alpes', 583],
                                   [2015, 'Auvergne-Rhône-Alpes', 593],
                                   [2016, 'Auvergne-Rhône-Alpes', 568],
                                   [2017, 'Auvergne-Rhône-Alpes', 735],
                                   [2018, 

In [11]:
fig.data

,Unnamed: 0.1,Unnamed: 0,Num_Acc,date,an,mois,jour,hrmn,dep,com,...,_infos_commune.code_epci,code_postal,nom_commune_postal,latitude,longitude,nom_commune,nom_commune_complet,num_dep,dep_name,region_name
0,0,0,200500000030,2005-01-13,2005,janvier,jeudi,19:45,62,62331,...,200033579.0,NaN,NaN,NaN,NaN,non renseigné,NaN,62,Pas-de-Calais,Hauts-de-France
1,1,1,200500000034,2005-01-19,2005,janvier,mercredi,10:45,62,62022,...,200018083.0,NaN,NaN,NaN,NaN,non renseigné,NaN,62,Pas-de-Calais,Hauts-de-France
2,2,2,200500000078,2005-01-26,2005,janvier,mercredi,13:15,02,02173,...,200071785.0,NaN,NaN,NaN,NaN,non renseigné,NaN,02,Aisne,Hauts-de-France
3,3,3,200500000093,2005-01-03,2005,janvier,lundi,13:30,02,02810,...,200071991.0,NaN,NaN,NaN,NaN,non renseigné,NaN,02,Aisne,Hauts-de-France
4,4,4,200500000170,2005-01-29,2005,janvier,samedi,18:30,76,76196,...,200084952.0,NaN,NaN,NaN,NaN,non renseigné,NaN,76,Seine-Maritime,Normandie
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94044,94044,94044,202100056508,2021-01-01,2021,janvier,vendredi,12:20,64,64400,...,246401756.0,64400.0,POEY D OLORON,43.246755,-0.663214,Poey-d'Oloron,Poey-d'Oloron,64,Pyrénées-Atlantiques,Nouvelle-Aquitaine
94045,94045,94045,202100056508,2021-01-01,2021,janvier,vendredi,12:20,64,64400,...,246401756.0,64400.0,PRECILHON,43.195778,-0.572974,Précilhon,Précilhon,64,Pyrénées-Atlantiques,Nouvelle-Aquitaine
94046,94046,94046,202100056508,2021-01-01,2021,janvier,vendredi,12:20,64,64400,...,246401756.0,64400.0,ST GOIN,43.231201,-0.725099,Saint-Goin,Saint-Goin,64,Pyrénées-Atlantiques,Nouvelle-Aquitaine
94047,94047,94047,202100056508,2021-01-01,2021,janvier,vendredi,12:20,64,64400,...,246401756.0,64400.0,SAUCEDE,43.265674,-0.669442,Saucède,Saucède,64,Pyrénées-Atlantiques,Nouvelle-Aquitaine
